In [1]:
import pandas as pd
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import win_unicode_console
win_unicode_console.enable()

class HangulLoader:
   
    def GetImages_Labels(self,  imgpath,labelpath ):
        #labels
        labels = pd.read_csv( labelpath , sep = ',' ).iloc[:,1]
        list_labels = list(labels)
        set_labels = list(set(list_labels))
        one_hot_vectors = np.eye(len(set_labels), dtype=int)
        class_vectors = {}
        for i, cls in enumerate(set_labels):
            class_vectors[cls] = one_hot_vectors[i]
        yList = [ class_vectors[cate_name] for cate_name in list_labels]
        yList = np.asarray(yList)
        classnum = len(set_labels)
        #images 
        file_list = []
        for root,_,fnames in os.walk(imgpath):
            fnames.sort(key = (lambda x : int(x.split('_')[1].split('.')[0])))
            for filename in fnames:
                file_list.append(os.path.join(root, filename))
        imglist = [ cv2.imread(path) for path in file_list]

        return imglist , yList , classnum










C:\Anaconda3\envs\tf\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == '949', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [2]:
#from Data_Loader import HangulLoader as ld

import os
import keras
from keras.layers import *
from keras import models , applications , callbacks , optimizers , losses
import numpy as np
from sklearn.model_selection import train_test_split
from functools import *

from Data_Loader import HangulLoader as hanLoader

import win_unicode_console
win_unicode_console.enable()

import keras.backend as K

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))



# data x ,y 
labelspath = r"data\hangul-images-data\labels-map.csv"
dirpath = r"data\hangul-images-data\img"
xs , ys , n_cls = hanLoader().GetImages_Labels( dirpath, labelspath )
# load model 

deepFeature = applications.VGG16(include_top = False , input_shape = (64,64,3) )

input = deepFeature.input

h = deepFeature.output
flat = Flatten()
dense1 = Dense( 4096 , activation = "relu" )
dense2 = Dense( 3096 , activation = "sigmoid" )
lastLayer = Dense( n_cls , activation = "sigmoid"  )

classifier = [ h , flat , dense1 , dense2 , lastLayer  ]

output = reduce( lambda f,s : s(f) , classifier)


C:\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda3\envs\tf\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == '949', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [3]:
model_cnn = models.Model( input , output )

model_cnn.compile( loss = losses.categorical_crossentropy , optimizer = optimizers.Adam() , matrics = ['acc'] )


hist = model_cnn.fit( xs , ys , batch_size = 32 , epochs  = 10 , validation_split = 0.2 )

print()




ValueError: Input arrays should have the same number of samples as target arrays. Found 18800 input samples and 18799 target samples.